> NOTE: To try this, you MUST be logged to Azure using PowerShell

Import d3js and its dependencies

In [ ]:
#r "nuget: Plotly.NET, 4.2.0"
#r "nuget: Plotly.NET.Interactive, 4.2.0"
#r "nuget: FSharp.Data"

In [ ]:
# some credentials + parameters
$WorkspaceID = '36d31d9f-bcbf-463b-9264-35006800c941' # some random GUID, just put your own
$dateFilter = 'ago(3d)'

Parameters and queries

In [ ]:
# queries and helper functions
$appQuery = @"
AZFWApplicationRule
| where TimeGenerated >= $dateFilter
| summarize Value = count() by Source = SourceIp, Target = Fqdn, Action
"@

$networkQuery = @"
AZFWNetworkRule | where TimeGenerated  >= $dateFilter
"@

$natQuery = @"
let Query1 =
    AZFWNatRule
    | where TimeGenerated >= $dateFilter
    | project SourceIp, Target = strcat_delim(":", DestinationIp, DestinationPort), Action = "Allow"
    | summarize Value = count() by Source = SourceIp, Target, Action;

let Query2 =
    AZFWNatRule
    | where TimeGenerated >= $dateFilter
    | project SourceStr = strcat_delim(":", DestinationIp, DestinationPort), TranslatedIp, Action = "Allow"
    | summarize Value = count() by Source = SourceStr, Target = TranslatedIp, Action;

union Query1, Query2
"@

Get Application-level data

In [ ]:
$appDataSet = Invoke-AzOperationalInsightsQuery -WorkspaceId $WorkspaceID -Query $appQuery -ErrorAction Stop | Select-Object -ExpandProperty Results

Get Network-level data

In [ ]:
$natDataSet = Invoke-AzOperationalInsightsQuery -WorkspaceId $WorkspaceID -Query $natQuery -ErrorAction Stop | Select-Object -ExpandProperty Results

This pulls data from Az Resource Graph on per-element basis, so takes time to execute

In [ ]:
# some additional data, pulling from Az Resource Graph
$subnets = 
($appDataSet| Select-Object -Unique -ExpandProperty Source) | % {
    $ipToSearch = $_
    $azGraphQueryExact = @"
resources
| where type == "microsoft.network/virtualnetworks"
| extend subnets = properties.subnets
| mv-expand subnet = subnets
| project vnetName = name,
        vnetId = id,
        vnetIpRange = properties.addressSpace.addressPrefixes[0],
        subnetName = subnet.name,
        subnetId = tostring(subnet.id),
        subnetIpRange = subnet.properties.addressPrefix
| extend subnetPath = strcat_delim("/", vnetName, subnetName)
| extend containingSubnet = ipv4_is_match('$ipToSearch', tostring(subnetIpRange) )
| where containingSubnet == 1
"@
    $subnetData = Search-AzGraph -Query $azGraphQueryExact
    [pscustomobject]@{
        Source = $subnetData.subnetPath
        Target = $_
        Action = "Allow"
        Value = 1
    }
}

Export data to `CSV` to simplify transferring it to F#. 

> NOTE: Please pay attention to the path.

In [ ]:
($appDataSet +  $subnets) | export-csv c:\temp\ds.csv
$natDataSet | export-csv c:\temp\nat-ds.csv

Prepare some processing functions and import data from `CSV`.

> NOTE. Please pay attention to the path.

In [ ]:
// import data
open FSharp.Data
open Plotly.NET
open System.IO
open Plotly.NET.ConfigObjects


let createNodeIndexMap (edges: (string * string * string * int) list) =
    edges
    |> Seq.collect (fun (src, tgt, _, _) -> [src; tgt])
    |> Seq.distinct
    |> Seq.mapi (fun i node -> (node, i))
    |> Map.ofSeq

let createIndexedEdges (edges: (string * string * string * int) list) (map: Map<string, int>) =
    let indexedEdges = edges |> List.map (fun (src, tgt, _, _) -> (map.[src], map.[tgt]))
    let weights = edges |> List.map (fun (_, _, _, weight) -> weight)
    (indexedEdges, weights)


let actionToColor action = 
    match action with
    | "Deny" -> Color.fromKeyword(ColorKeyword.LightCoral)
    | _ -> Color.fromKeyword(ColorKeyword.LightGray)

type commsDataType = CsvProvider<"c:\\temp\\ds.csv">
type natDataType = CsvProvider<"c:\\temp\\nat-ds.csv">

let commsData = commsDataType.Load("c:\\temp\\ds.csv")
let natData = natDataType.Load("c:\\temp\\nat-ds.csv")

Comms processing: preparing data and visualizing

In [ ]:
// prepare data + some anonymization (for video recording)

open System.Text.RegularExpressions

let anonymizeToggle = false // Set to false to disable anonymization. In case you decide to record a video ;)
let namesToHide = ["some";"values";]

let anonymizeSubstrings (input: string) =
    namesToHide
    |> List.fold (fun (acc: String) name -> acc.Replace(name, "xxx")) input

let anonymizeIp (input: string) =
    match input with
    | ip when Regex.IsMatch(ip, @"\b\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}\b") ->
        // IP address found, anonymize middle octets
        let parts = ip.Split('.')
        sprintf "%s.x.x.%s" parts.[0] parts.[3]
    | domain when Regex.IsMatch(domain, @"^[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}$") ->
        // Domain name found, replace specified subdomains with 'x'
        let parts = domain.Split('.')
        let anonymizedSubdomains = parts
                                    |> Array.map anonymizeSubstrings
        sprintf "%s" (String.Join(".", anonymizedSubdomains))
    | otherwise ->
        // No action needed, return as is
        otherwise


let edges = 
    commsData.Rows
    |> Seq.map (fun row -> 
        let source = row.Source
        let target = row.Target
        let action = row.Action
        let weight = row.Value
        (source, target, action, weight)
    ) |> Seq.toList

let nodeIndexMap = createNodeIndexMap edges
let (indexedEdges, weights) = createIndexedEdges edges nodeIndexMap

let nodeList = nodeIndexMap 
                |> Map.toList 
                |> List.sortBy snd 
                |> List.map fst 
                |> Seq.map (fun row ->
                    if anonymizeToggle then anonymizeIp row else row
                )
let edgesArray = indexedEdges |> List.toArray
let weightsArray = weights |> List.toArray

let linkColors = edges |> List.map (fun (_, _, action, _) -> actionToColor action)

In [ ]:
// visualize

let height = 1500
let width = 1500

let sankey1 =
    Chart.Sankey(
        nodeLabels = nodeList,
        linkedNodeIds = edgesArray,
        linkValues = weightsArray,
        LinkColor = Color.fromColors(linkColors)
    )

let svgConfig =
    Config.init (
    ToImageButtonOptions =
        ToImageButtonOptions.init (
            Format = StyleParam.ImageFormat.JPEG,
            Filename = "myChart",
            Width = width,
            Height = height,
            Scale = 10.
        )
)

let updated = 
    sankey1
    |> Chart.withLayout(Layout.init(Width=width, Height = height))
    |> Chart.withConfig(svgConfig)

updated